In [1]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.1 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2


In [2]:
import torch
print(f"Base PyTorch: {torch.__version__}")

Base PyTorch: 2.8.0+cu128


In [3]:
!pip install "unsloth[cu128-torch280] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-xksp_2k8/unsloth_ee7f7dbc56d14d38baebd4e354d77871
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-xksp_2k8/unsloth_ee7f7dbc56d14d38baebd4e354d77871
  Resolved https://github.com/unslothai/unsloth.git to commit 8ea5338154859ed25b50366cb1264ed4d933eae3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 MB 34.4 MB/s  0:00:03:00:0100:01
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.1 MB/s  0:00:04 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 25.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 16.9 MB/s  0:00

In [4]:
!pip install xformers trl peft accelerate bitsandbytes

In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # Auto detection
load_in_4bit = True 

print("⬇️  Downloading Llama 3 8B...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ Model Loaded. Attaching LoRA Adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)
print("🚀 Ready for Data.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
⬇️  Downloading Llama 3 8B...
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.527 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

✅ Model Loaded. Attaching LoRA Adapters...


Unsloth 2025.12.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


🚀 Ready for Data.


In [6]:
from datasets import load_dataset

# 1. Define the "Alpaca" Prompt Template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Critical: Tells the model where to stop generating.

# 2. Formatting Function
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Fill the template + Append EOS token
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 3. Load Data
# Ensure the filename matches exactly what you uploaded
dataset = load_dataset("json", data_files="physician_style_dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

print(f"✅ Data Formatted. Training on {len(dataset)} records.")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/491 [00:00<?, ? examples/s]

✅ Data Formatted. Training on 491 records.


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 1 Epoch (approx)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("🔥 Initiating Training Sequence...")
trainer_stats = trainer.train()
print("✅ Training Complete.")

Unsloth: Tokenizing ["text"] (num_proc=260):   0%|          | 0/491 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🔥 Initiating Training Sequence...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 491 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.868400
2,1.842300
3,1.813400
4,1.777500
5,1.738000
6,1.638000
7,1.488000
8,1.407200
9,1.372400
10,1.287300


✅ Training Complete.


In [10]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Define the test case
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a physician's assistant drafting a reply to a patient's email. Be professional, empathetic, and cautious. Your name is Henry Smith and your practice name is Hempstead Cardiology Associates", # Instruction
        "Hi Dr. Smith, I forgot to take my heart medication (Lisinopril) yesterday. I'm worried. Should I take two pills today to make up for it? - Jeremiah Jones", # Input
        "", # Output - leave blank for generation
    )
], return_tensors = "pt").to("cuda")

# Generate
print("Generating Reply...")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
result = tokenizer.batch_decode(outputs)

# Clean up the output to show just the response
response_text = result[0].split("### Response:\n")[-1].replace(EOS_TOKEN, "")
print("\n" + "="*20 + " MODEL OUTPUT " + "="*20)
print(response_text)
print("="*54)

Generating Reply...

==================== MODEL OUTPUT ====================
Dear Mr. Jones,

Thank you for reaching out about your missed Lisinopril dose. I understand that missing a medication can be concerning, especially when it's for heart health.

For missed doses of Lisinopril, the general recommendation is to take the missed dose as soon as possible if it's within 6-8 hours of your usual time. If it's been more than 8 hours, just skip that dose and continue with your regular schedule. Don't double up on doses or take extra pills.

If you're experiencing any symptoms like chest pain, shortness of breath, dizziness, or your blood pressure is unusually high or low, please contact us immediately or go to the emergency room.

If you're concerned about the impact of this missed dose, please call our office at [phone number] to discuss your specific situation.

You can also find more information about missed doses in our patient portal under "Medication Management."

We're here to help

In [12]:
# Save to GGUF format (Quantized for Mac/Local use)
# We use q4_k_m (4-bit medium) which is the industry standard balance of speed/quality.

print("💾 Converting and Saving GGUF...")
model.save_pretrained_gguf("docmail_llama3_model", tokenizer, quantization_method = "q4_k_m")
print("✅ Conversion Complete.")

💾 Converting and Saving GGUF...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /workspace/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 19043.38it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:31<00:00,  7.91s/it]


Unsloth: Merge process complete. Saved to `/workspace/docmail_llama3_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: llama.cpp folder exists but binaries not found - will rebuild
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Install GGUF and other packages


RuntimeError: Unsloth: GGUF conversion failed: [FAIL] Command `uv pip install gguf protobuf sentencepiece mistral_common` failed with exit code 2
stdout: [1m[31merror[39m[0m: No virtual environment found; run `[32muv venv[39m` to create an environment, or pass `[32m--system[39m` to install into a non-virtual environment



In [14]:
# 1. Manually install the Python dependencies that 'uv' choked on
!pip install gguf protobuf sentencepiece mistral_common

# 2. Force compile llama.cpp (Ensure the 'quantize' binary exists)
# This fixes the "No working quantizer found" error at the top of your log.
%cd llama.cpp
!make clean
!make all -j
%cd .. 

# 3. Verify the binary exists
import os
if os.path.exists("llama.cpp/llama-quantize"):
    print("✅ Compilation Successful. Quantizer binary found.")
else:
    print("❌ Compilation Failed.")

/workspace/llama.cpp
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.
/workspace
❌ Compilation Failed.


In [15]:
# 1. Ensure build tools are present
!pip install cmake gguf protobuf sentencepiece mistral_common

# 2. Compile llama.cpp using CMake
%cd llama.cpp
!rm -rf build
!mkdir build
# We build Release mode. We skip CUDA for quantization to avoid nvcc version headaches (CPU is fast enough for conversion).
!cd build && cmake .. -DGGML_CUDA=OFF && cmake --build . --config Release -j --target llama-quantize

# 3. Deployment: Move the binary to the root folder where Unsloth looks
# CMake puts binaries in build/bin/. We copy it up.
!cp build/bin/llama-quantize .

# 4. Compatibility Shim: Unsloth might look for the old name 'quantize'
!cp llama-quantize quantize

%cd ..

# 5. Verification
import os
if os.path.exists("llama.cpp/llama-quantize"):
    print("✅ CMake Build Successful. Binary located.")
else:
    print("❌ Build failed. Check logs above.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 1.1 MB/s  0:00:27m0:00:010:00:03
/workspace/llama.cpp
-- The C compiler identification is GNU 13.3.0
-- The CXX compiler identification is GNU 13.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.43.0")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCA

In [16]:
# Save to GGUF format (Quantized for Mac/Local use)
# We use q4_k_m (4-bit medium) which is the industry standard balance of speed/quality.

print("💾 Converting and Saving GGUF...")
model.save_pretrained_gguf("docmail_llama3_model", tokenizer, quantization_method = "q4_k_m")
print("✅ Conversion Complete.")

💾 Converting and Saving GGUF...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /workspace/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 10242.50it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:30<00:00,  7.74s/it]


Unsloth: Merge process complete. Saved to `/workspace/docmail_llama3_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3-8b.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['llama-3-8b.Q4_K_M.gguf']
Unsloth: No Ollama template mapping found for model 'unsloth/llama-3-8b'. Skipping Ol